<a href="https://colab.research.google.com/github/john-d-noble/callcenter/blob/main/XGB_High_Performance_Model_Final_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install yfinance

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.2/949.2 kB 15.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 109.0 MB/s eta 0:00:00
  Created wheel for multitasking: filename=multitasking-0.0.12-py3-none-any.whl size=15636 sha256=925987fb7f0e47a673e515b6221906848bf9cdfd287be397457bb4951ebcffed
  Stored in directory: /root/.cache/pip/wheels/cc/bd/6f/664d62c99327abeef7d86489e6631cbf45b56fbf7ef1d6ef00
  Created wheel for peewee: filename=peewee-3.18.2-cp312-cp312-linux_x86_64.whl size=936670 sha256=55782272e0deb9e2e573fd2232755b0c2e776e02fdae7f973dd4372f0137234a
  Stored in directory: /root/.cache/pip/wheels/d1/df/a9/0

In [2]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 MB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.5/322.5 MB 3.3 MB/s eta 0:00:00


In [3]:
#
# MASTER SCRIPT: From Data Collection to Live Forecasting (Version 3.1 - With Validation Pause)
# This script combines all steps into a single, sequential process with robust data cleaning.
#

import pandas as pd
import numpy as np
import yfinance as yf
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV, TimeSeriesSplit
from datetime import date, timedelta

# --- MASTER CONFIGURATION ---
# Data Collection & Simulation
TICKERS = ['BTC-USD', 'ETH-USD', 'SOL-USD', '^VIX']
START_DATE = '2021-01-01'

# Feature Engineering
VOLATILITY_WINDOW = 14
SPIKE_WINDOW = 30
SPIKE_THRESHOLD = 2.0

# Model Tuning
TUNING_ITERATIONS = 50 # Number of hyperparameter combinations to test

# Final Output Files
MARKET_DATA_FILE = 'crypto_price_and_volume_data_2021_present.csv'
SIMULATED_CALLS_FILE = 'final_simulated_data_with_volatility_2021_present.csv'
ADVANCED_TRAINING_FILE = 'advanced_feature_training_data_2021_present.csv'
FINAL_MODEL_FILENAME = 'final_advanced_xgboost_model_2021_present.json'


# ==============================================================================
# STEP 1: DATA COLLECTION - Fetch all required price and volume data
# ==============================================================================
print("--- [STEP 1/7] Starting: Data Collection ---")
try:
    END_DATE = date.today().strftime('%Y-%m-%d')
    raw_data = yf.download(TICKERS, start=START_DATE, end=END_DATE)

    # Create a complete daily date range to ensure no gaps
    full_date_range = pd.date_range(start=START_DATE, end=END_DATE, freq='D')
    market_data_df = pd.DataFrame(index=full_date_range)

    # Define a mapping from ticker to a clean name for columns
    TICKER_MAP = {
        'BTC-USD': 'btc', 'ETH-USD': 'eth',
        'SOL-USD': 'sol', '^VIX': 'vix'
    }

    for ticker, name in TICKER_MAP.items():
        price_series = raw_data[('Close', ticker)]
        volume_series = raw_data[('Volume', ticker)]
        asset_df = pd.DataFrame({f'{name}_price': price_series, f'{name}_volume': volume_series})
        asset_df = asset_df.reindex(full_date_range).ffill().bfill()
        market_data_df = market_data_df.join(asset_df)

    market_data_df.index.name = 'Date'
    market_data_df.to_csv(MARKET_DATA_FILE)
    print(f"✅ Success! Market data saved to '{MARKET_DATA_FILE}'")

    # --- MODIFICATION START: Added a data validation and pause point ---
    print("\n--- [DATA REVIEW & VALIDATION] ---")
    print(f"Date Range: {market_data_df.index.min().strftime('%Y-%m-%d')} to {market_data_df.index.max().strftime('%Y-%m-%d')}")
    print(f"Total Days (Rows) Collected: {market_data_df.shape[0]}")
    print(f"Total Features (Columns): {market_data_df.shape[1]}")
    print("\n--- Missing Values Check (should all be 0) ---")
    print(market_data_df.isnull().sum())

    input("\n--- PAUSED --- \nReview the counts above. Press Enter to proceed to Step 2 or Ctrl+C to exit.\n")
    # --- MODIFICATION END ---

except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 2: SIMULATION - Create the intelligent, volatility-driven call volume
# ==============================================================================
print(f"--- [STEP 2/7] Starting: Call Volume Simulation ---")
try:
    df_sim = pd.read_csv(MARKET_DATA_FILE, parse_dates=['Date'])

    # Baseline simulation parameters
    NUMBER_OF_AGENTS = 3000
    AVG_WEEKDAY_CALLS_PER_AGENT = 110
    AVG_WEEKEND_CALLS_PER_AGENT = 50
    RANDOMNESS_FACTOR = 15000

    def simulate_base_volume(d):
        mean_vol = NUMBER_OF_AGENTS * (AVG_WEEKDAY_CALLS_PER_AGENT if d.dayofweek < 5 else AVG_WEEKEND_CALLS_PER_AGENT)
        return abs(int(np.random.normal(loc=mean_vol, scale=RANDOMNESS_FACTOR)))
    df_sim['base_call_volume'] = df_sim['Date'].apply(simulate_base_volume)

    # Volatility adjustment parameters
    VOLATILITY_THRESHOLD = 0.05
    VOLATILITY_MULTIPLIER = 1.8
    VIX_FEAR_THRESHOLD = 30
    VIX_MULTIPLIER = 1.4

    for crypto in ['btc', 'eth', 'sol']:
        df_sim[f'{crypto}_price_pct_change'] = df_sim[f'{crypto}_price'].pct_change()

    def adjust_volume_for_volatility(row):
        final_volume = row['base_call_volume']
        crypto_volatile = any(abs(row[f'{c}_price_pct_change']) > VOLATILITY_THRESHOLD for c in ['btc', 'eth', 'sol'])
        vix_high = row['vix_price'] > VIX_FEAR_THRESHOLD
        if crypto_volatile: final_volume *= VOLATILITY_MULTIPLIER
        if vix_high: final_volume *= VIX_MULTIPLIER
        return int(final_volume)

    df_sim.fillna(0, inplace=True)
    df_sim['adjusted_call_volume'] = df_sim.apply(adjust_volume_for_volatility, axis=1)

    simulated_df = df_sim[['Date', 'btc_price', 'eth_price', 'sol_price', 'vix_price', 'adjusted_call_volume']]
    simulated_df.to_csv(SIMULATED_CALLS_FILE, index=False)
    print(f"✅ Success! Simulated call volume data saved to '{SIMULATED_CALLS_FILE}'\n")
except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 3: ADVANCED FEATURE ENGINEERING - Create the final training data
# ==============================================================================
print(f"--- [STEP 3/7] Starting: Advanced Feature Engineering ---")
try:
    df_market = pd.read_csv(MARKET_DATA_FILE, parse_dates=['Date'])
    df_calls = pd.read_csv(SIMULATED_CALLS_FILE, parse_dates=['Date'])

    for crypto in ['btc', 'eth', 'sol']:
        df_market[f'{crypto}_price_pct_change'] = df_market[f'{crypto}_price'].pct_change()
        df_market[f'{crypto}_volatility_index'] = df_market[f'{crypto}_price_pct_change'].rolling(window=VOLATILITY_WINDOW).std()

        rolling_vol_mean = df_market[f'{crypto}_volume'].rolling(window=SPIKE_WINDOW).mean()
        rolling_vol_std = df_market[f'{crypto}_volume'].rolling(window=SPIKE_WINDOW).std()
        df_market[f'{crypto}_volume_spike'] = (df_market[f'{crypto}_volume'] > (rolling_vol_mean + SPIKE_THRESHOLD * rolling_vol_std)).astype(int)

        rolling_price_mean = df_market[f'{crypto}_price_pct_change'].rolling(window=SPIKE_WINDOW).mean()
        rolling_price_std = df_market[f'{crypto}_price_pct_change'].rolling(window=SPIKE_WINDOW).std()
        df_market[f'{crypto}_price_shock'] = (abs(df_market[f'{crypto}_price_pct_change']) > (rolling_price_mean + SPIKE_THRESHOLD * rolling_price_std)).astype(int)

    df_calls_subset = df_calls[['Date', 'adjusted_call_volume']]
    combined_df = pd.merge(df_market, df_calls_subset, on='Date', how='inner')
    combined_df.drop(columns=[col for col in combined_df.columns if 'pct_change' in col], inplace=True)
    combined_df.fillna(0, inplace=True)
    combined_df.to_csv(ADVANCED_TRAINING_FILE, index=False)
    print(f"✅ Success! Advanced training file saved as '{ADVANCED_TRAINING_FILE}'\n")
except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 4: MODEL TUNING - Find the best hyperparameters
# ==============================================================================
print(f"--- [STEP 4/7] Starting: Automated Model Tuning (this may take several minutes) ---")
try:
    df_tune = pd.read_csv(ADVANCED_TRAINING_FILE, parse_dates=['Date'])
    df_tune.set_index('Date', inplace=True)
    df_tune.fillna(0, inplace=True)
    X = df_tune.drop('adjusted_call_volume', axis=1)
    y = df_tune['adjusted_call_volume']

    param_grid = {
        'n_estimators': [100, 300, 500, 700], 'max_depth': [3, 4, 5, 6, 7],
        'learning_rate': [0.01, 0.05, 0.1], 'subsample': [0.7, 0.8, 0.9, 1.0],
        'colsample_bytree': [0.7, 0.8, 0.9, 1.0]
    }
    xgb_model = xgb.XGBRegressor(objective='reg:squarederror', eval_metric='mae')
    tscv = TimeSeriesSplit(n_splits=5)
    random_search = RandomizedSearchCV(estimator=xgb_model, param_distributions=param_grid, n_iter=TUNING_ITERATIONS,
                                   scoring='neg_mean_absolute_error', cv=tscv, n_jobs=-1, verbose=1, random_state=42)
    random_search.fit(X, y)

    best_params = random_search.best_params_
    best_score = -random_search.best_score_

    print("\n--- Tuning Complete ---")
    print(f"✅ Best MAE Score found: {best_score:.2f}")
    print(f"✅ Best Hyperparameters found: {best_params}\n")
except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 5: FINAL MODEL TRAINING - Build the model with the best settings
# ==============================================================================
print(f"--- [STEP 5/7] Starting: Final Model Training ---")
try:
    final_model = xgb.XGBRegressor(
        objective='reg:squarederror',
        eval_metric='mae',
        **best_params,
        random_state=42
    )
    final_model.fit(X, y)
    print("✅ Success! Final model trained on all available data.\n")
except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 6: SAVE THE MODEL - Create the final, predictive asset
# ==============================================================================
print(f"--- [STEP 6/7] Starting: Saving the Final Model ---")
try:
    final_model.save_model(FINAL_MODEL_FILENAME)
    print(f"✅ Success! Final model has been saved as '{FINAL_MODEL_FILENAME}'.\n")
except Exception as e:
    print(f"❌ FAILED: {e}")
    exit()

# ==============================================================================
# STEP 7: MAKE A PREDICTION - Use the model for a live forecast
# ==============================================================================
print(f"--- [STEP 7/7] Starting: Making a Live Forecast ---")
try:
    forecast_start_date = date.today() - timedelta(days=35)
    forecast_end_date = date.today()
    raw_forecast_data = yf.download(TICKERS, start=forecast_start_date, end=forecast_end_date, progress=False)

    forecast_full_range = pd.date_range(start=forecast_start_date, end=forecast_end_date, freq='D')
    market_df = pd.DataFrame(index=forecast_full_range)

    for ticker, name in TICKER_MAP.items():
        price_series = raw_forecast_data[('Close', ticker)]
        volume_series = raw_forecast_data[('Volume', ticker)]
        asset_df = pd.DataFrame({f'{name}_price': price_series, f'{name}_volume': volume_series})
        asset_df = asset_df.reindex(forecast_full_range).ffill().bfill()
        market_df = market_df.join(asset_df)

    market_df.index.name = 'Date'

    for crypto in ['btc', 'eth', 'sol']:
        market_df[f'{crypto}_price_pct_change'] = market_df[f'{crypto}_price'].pct_change()
        market_df[f'{crypto}_volatility_index'] = market_df[f'{crypto}_price_pct_change'].rolling(window=VOLATILITY_WINDOW).std()
        rolling_vol_mean = market_df[f'{crypto}_volume'].rolling(window=SPIKE_WINDOW).mean()
        rolling_vol_std = market_df[f'{crypto}_volume'].rolling(window=SPIKE_WINDOW).std()
        market_df[f'{crypto}_volume_spike'] = (market_df[f'{crypto}_volume'] > (rolling_vol_mean + SPIKE_THRESHOLD * rolling_vol_std)).astype(int)
        rolling_price_mean = market_df[f'{crypto}_price_pct_change'].rolling(window=SPIKE_WINDOW).mean()
        rolling_price_std = market_df[f'{crypto}_price_pct_change'].rolling(window=SPIKE_WINDOW).std()
        market_df[f'{crypto}_price_shock'] = (abs(market_df[f'{crypto}_price_pct_change']) > (rolling_price_mean + SPIKE_THRESHOLD * rolling_price_std)).astype(int)

    prediction_input = market_df.tail(1)
    training_columns = [col for col in X.columns]
    prediction_input = prediction_input[training_columns]

    loaded_model = xgb.XGBRegressor()
    loaded_model.load_model(FINAL_MODEL_FILENAME)

    prediction = loaded_model.predict(prediction_input)
    predicted_volume = int(prediction[0])
    latest_day = prediction_input.index[0]

    print("\n--- Forecast Complete ---")
    print(f"✅ Based on the latest market data for {latest_day.strftime('%Y-%m-%d')}, the model forecasts a call volume of: {predicted_volume:,}")
except Exception as e:
    print(f"❌ FAILED: {e}")

--- [STEP 1/7] Starting: Data Collection ---


/tmp/ipython-input-1517842121.py:39: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_data = yf.download(TICKERS, start=START_DATE, end=END_DATE)
[*********************100%***********************]  4 of 4 completed


✅ Success! Market data saved to 'crypto_price_and_volume_data_2021_present.csv'

--- [DATA REVIEW & VALIDATION] ---
Date Range: 2021-01-01 to 2025-08-25
Total Days (Rows) Collected: 1698
Total Features (Columns): 8

--- Missing Values Check (should all be 0) ---
btc_price     0
btc_volume    0
eth_price     0
eth_volume    0
sol_price     0
sol_volume    0
vix_price     0
vix_volume    0
dtype: int64

--- PAUSED --- 
Review the counts above. Press Enter to proceed to Step 2 or Ctrl+C to exit.

--- [STEP 2/7] Starting: Call Volume Simulation ---
✅ Success! Simulated call volume data saved to 'final_simulated_data_with_volatility_2021_present.csv'

--- [STEP 3/7] Starting: Advanced Feature Engineering ---
✅ Success! Advanced training file saved as 'advanced_feature_training_data_2021_present.csv'

--- [STEP 4/7] Starting: Automated Model Tuning (this may take several minutes) ---
Fitting 5 folds for each of 50 candidates, totalling 250 fits

--- Tuning Complete ---
✅ Best MAE Score found

/tmp/ipython-input-1517842121.py:219: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_forecast_data = yf.download(TICKERS, start=forecast_start_date, end=forecast_end_date, progress=False)



--- Forecast Complete ---
✅ Based on the latest market data for 2025-08-25, the model forecasts a call volume of: 320,071
